In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import os
import string

In [ ]:
PATH='/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/medical_dataset'
dataset=['train.csv','valid.csv','test.csv']

In [ ]:

def lowercase(corpus):
  corpus['text']=corpus['text'].str.lower()
  return corpus

def special_element_removal(corpus):
  clean = re.compile('\d\.')
  corpus["text"] = corpus["text"].apply(lambda x: clean.sub(' ',x)) #numbering of patient symptoms are removed
  return corpus

def tokenize(corpus):
  clean = re.compile('[^a-zA-Z0-9]')
  corpus["text"] = corpus["text"].apply(lambda x: clean.sub(' ',x))
  clean = re.compile('[\s\t\n]+')
  corpus["words"] = corpus["text"].apply(lambda x: re.split('[\s\t\n]+',x))
  return corpus


In [ ]:

file_path=os.path.join(PATH, dataset[0])
train_df=pd.read_csv(file_path)
# print(f'{data.split(".")[0]}:\n{df.head()}')
train_df=lowercase(train_df)
train_df=special_element_removal(train_df)
train_df=tokenize(train_df)


In [ ]:
file_path=os.path.join(PATH, dataset[1])
valid_df=pd.read_csv(file_path)
# print(f'{data.split(".")[0]}:\n{df.head()}')
valid_df=lowercase(valid_df)
valid_df=special_element_removal(valid_df)
valid_df=tokenize(valid_df)

In [ ]:
file_path=os.path.join(PATH, dataset[2])
test_df=pd.read_csv(file_path)
# print(f'{data.split(".")[0]}:\n{df.head()}')
test_df=lowercase(test_df)
test_df=special_element_removal(test_df)
test_df=tokenize(test_df)

In [ ]:
#N frequent word in corpus
def frequent_words(corpus_df,N):
  tokens=[]
  for raw in corpus_df["words"]:
    for token in raw:
      tokens.append(token)
  bag_words={}
  for token in tokens:
    if token in bag_words:
      bag_words[token]+=1
    else:
      bag_words[token]=1
  df_freq=pd.DataFrame.from_dict(bag_words, orient='index',
                       columns=['frequency'])
  df_freq=df_freq.sort_values(by=['frequency'],ascending=False)
  df_freq.index.names = ['Word']
  df_freq=df_freq.reset_index()
  # df_freq.index += 1 
  df_freq['id']=df_freq.index+1 
  df_freq = df_freq[['Word','id','frequency']]
  return(df_freq.head(N))

In [ ]:
freq_words_training=frequent_words(train_df,10000)

In [ ]:
freq_words_training.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/medical_nlp-vocab.txt', header=True, index=None, sep='\t')

In [ ]:
freq_words_training=pd.read_csv('/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/medical_nlp-vocab.txt', sep='\t')
freq_words_training

,Word,id,frequency
0,the,1,118887
1,and,2,66917
2,was,3,56124
3,of,4,48447
4,to,5,41003
...,...,...,...
9995,radiolucent,9996,8
9996,l5s,9997,8
9997,proteins,9998,8
9998,equivalents,9999,8


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# train_df["joined_words"] = train_df.words.apply(lambda x: " ".join(x))
# CountVec = CountVectorizer(vocabulary=freq_words_training['Word'][:40],binary=True,tokenizer=word_tokenize)
# Count_data = CountVec.fit_transform(train_df["text"])
# feature_names = CountVec.get_feature_names()
# binary_bgw_sk=pd.DataFrame.sparse.from_spmatrix(Count_data,columns=feature_names)


In [ ]:
#this method calculate binary bag of word 
import tqdm
def binary_bgw(df,vocabulary):
  binary_bgw=[]
  nbr_word=len(vocabulary)
  for row in tqdm.tqdm(df["words"]):
    bg=[0]*nbr_word
    for i,word in enumerate(vocabulary):
      if word in row:
        bg[i]=1

    binary_bgw.append(bg)
  return binary_bgw



In [ ]:
binary_bgw_train=binary_bgw(train_df,list(freq_words_training['Word']))

100%|██████████| 4000/4000 [05:36<00:00, 11.87it/s]


In [ ]:
binary_bgw_train= pd.DataFrame(binary_bgw_train, columns=freq_words_training['Word'])
binary_bgw_train.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/binary-bgw_train.txt', header=True, index=None, sep='\t')

In [ ]:
binary_bgw_train=pd.read_csv('/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/binary-bgw_train.txt', sep='\t')


In [ ]:
binary_bgw_valid=binary_bgw(valid_df,list(freq_words_training['Word']))

100%|██████████| 499/499 [00:42<00:00, 11.82it/s]


In [ ]:
binary_bgw_valid= pd.DataFrame(binary_bgw_valid, columns=freq_words_training['Word'])
binary_bgw_valid.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/binary-bgw_valid.txt', header=True, index=None, sep='\t')

In [ ]:
binary_bgw_valid=pd.read_csv('/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/binary-bgw_valid.txt', sep='\t')

In [ ]:
binary_bgw_test=binary_bgw(test_df,list(freq_words_training['Word']))

100%|██████████| 500/500 [00:42<00:00, 11.67it/s]


In [ ]:
binary_bgw_test= pd.DataFrame(binary_bgw_test, columns=freq_words_training['Word'])
binary_bgw_test.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/binary-bgw_test.txt', header=True, index=None, sep='\t')

In [ ]:
binary_bgw_test=pd.read_csv('/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/binary-bgw_test.txt', sep='\t')

In [ ]:
from scipy.sparse import csr_matrix

binary_bgw_scrMatrix=csr_matrix(binary_bgw.values)
binary_bgw_scrMatrix

<4000x10000 sparse matrix of type '<class 'numpy.longlong'>'
	with 874180 stored elements in Compressed Sparse Row format>

In [ ]:
import tqdm
def freq_bgw(df,vocabulary):

  freq_bgw=[]
  # freq_word=list(freq_words_training['Word'])
  nbr_word=len(vocabulary)
  for row in tqdm.tqdm(df["words"]):
    bg=[0]*nbr_word
    for word in row:
      if word in vocabulary:
        bg[vocabulary.index(word)]+=1
    bg_fr=[x / sum(bg) for x in bg]
    freq_bgw.append(bg_fr)
  return freq_bgw

In [ ]:
#this method calculate frequency bag of word
# import tqdm
# freq_bgw=[]
# freq_word=list(freq_words_training['Word'])
# nbr_word=freq_words_training['Word'].shape[0]
# for row in tqdm.tqdm(train_df.iloc[0:3,:]["words"]):
#   bg=[0]*nbr_word
#   for word in row:
#     if word in freq_word:
#       bg[freq_words_training[freq_words_training['Word']==word].index.values.astype(int)[0]]+=1
#   bg_fr=[x / sum(bg) for x in bg]
#   freq_bgw.append(bg_fr)

# freq_bgw= pd.DataFrame(freq_bgw, columns=freq_words_training['Word'])
# freq_bgw.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw.txt', header=True, index=None, sep='\t')

100%|██████████| 3/3 [00:03<00:00,  1.13s/it]


In [ ]:
freq_bgw_train=freq_bgw(train_df,list(freq_words_training['Word']))
freq_bgw_train= pd.DataFrame(freq_bgw_train, columns=freq_words_training['Word'])
freq_bgw_train
freq_bgw_train.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw_train.txt', header=True, index=None, sep='\t')

100%|██████████| 4000/4000 [55:20<00:00,  1.20it/s]


In [ ]:
freq_bgw_train=pd.read_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw_train.txt', sep='\t')
freq_bgw_train

,the,and,was,of,to,a,with,in,is,patient,no,she,for,he,were,on,this,at,then,right,or,left,has,as,that,her,history,there,had,his,be,normal,procedure,not,an,placed,1,are,well,i,...,lorcet,atheromatous,wasting,somnolence,tremendous,phisohex,immigrated,itch,anticoagulated,stranded,durable,precarinal,100s,cognition,850,participates,condylectomy,serve,officer,monocular,hematopoietic,talofibular,178,conquer,hydrate,sealing,pericranial,glandular,subconjunctival,rinse,cousin,photodocumentation,emphasized,interaction,stabilizing,radiolucent,l5s,proteins,equivalents,suv
0,0.009804,0.019608,0.000000,0.009804,0.009804,0.000000,0.009804,0.019608,0.009804,0.000000,0.009804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.039216,0.000000,0.000000,0.000000,0.000000,0.000000,0.009804,0.000000,0.000000,0.000000,0.049020,0.000000,0.000000,0.000000,0.000000,0.009804,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.127208,0.031802,0.056537,0.031802,0.021201,0.014134,0.010601,0.003534,0.000000,0.003534,0.003534,0.000000,0.000000,0.000000,0.007067,0.000000,0.007067,0.017668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003534,0.000000,0.000000,0.007067,0.000000,0.000000,0.003534,0.024735,0.010601,0.003534,0.003534,0.003534,0.000000,0.000000,0.003534,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.016588,0.035545,0.004739,0.011848,0.014218,0.018957,0.009479,0.009479,0.037915,0.002370,0.014218,0.059242,0.011848,0.000000,0.000000,0.002370,0.002370,0.000000,0.002370,0.000000,0.004739,0.000000,0.028436,0.004739,0.011848,0.021327,0.004739,0.000000,0.009479,0.000000,0.002370,0.002370,0.000000,0.011848,0.000000,0.000000,0.002370,0.002370,0.004739,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.091575,0.042125,0.038462,0.021978,0.027473,0.012821,0.020147,0.010989,0.000000,0.020147,0.003663,0.000000,0.016484,0.000000,0.010989,0.001832,0.001832,0.009158,0.001832,0.000000,0.007326,0.000000,0.000000,0.005495,0.001832,0.000000,0.000000,0.000000,0.000000,0.000000,0.001832,0.001832,0.020147,0.000000,0.000000,0.001832,0.010989,0.000000,0.000000,0.001832,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.027569,0.032581,0.017544,0.050125,0.032581,0.007519,0.027569,0.007519,0.010025,0.005013,0.000000,0.020050,0.007519,0.000000,0.002506,0.020050,0.000000,0.010025,0.000000,0.000000,0.005013,0.002506,0.000000,0.000000,0.005013,0.017544,0.007519,0.000000,0.005013,0.000000,0.015038,0.000000,0.000000,0.000000,0.002506,0.000000,0.005013,0.002506,0.000000,0.002506,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.058333,0.033333,0.016667,0.008333,0.008333,0.000000,0.050000,0.025000,0.008333,0.025000,0.008333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,0.008333,0.025000,0.008333,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
freq_bgw_valid=freq_bgw(valid_df,list(freq_words_training['Word']))
freq_bgw_valid= pd.DataFrame(freq_bgw_valid, columns=freq_words_training['Word'])
freq_bgw_valid
freq_bgw_valid.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw_valid.txt', header=True, index=None, sep='\t')

100%|██████████| 499/499 [06:54<00:00,  1.20it/s]


In [ ]:
freq_bgw_valid=pd.read_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw_valid.txt', sep='\t')
freq_bgw_valid

,the,and,was,of,to,a,with,in,is,patient,no,she,for,he,were,on,this,at,then,right,or,left,has,as,that,her,history,there,had,his,be,normal,procedure,not,an,placed,1,are,well,i,...,lorcet,atheromatous,wasting,somnolence,tremendous,phisohex,immigrated,itch,anticoagulated,stranded,durable,precarinal,100s,cognition,850,participates,condylectomy,serve,officer,monocular,hematopoietic,talofibular,178,conquer,hydrate,sealing,pericranial,glandular,subconjunctival,rinse,cousin,photodocumentation,emphasized,interaction,stabilizing,radiolucent,l5s,proteins,equivalents,suv
0,0.082090,0.044776,0.007463,0.022388,0.007463,0.007463,0.000000,0.007463,0.022388,0.007463,0.000000,0.000000,0.007463,0.007463,0.000000,0.052239,0.014925,0.000000,0.000000,0.014925,0.000000,0.022388,0.007463,0.000000,0.000000,0.000000,0.000000,0.007463,0.000000,0.000000,0.000000,0.000000,0.000000,0.007463,0.000000,0.000000,0.000000,0.000000,0.000000,0.007463,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.041667,0.026882,0.016129,0.016129,0.022849,0.016129,0.010753,0.017473,0.013441,0.006720,0.012097,0.016129,0.006720,0.000000,0.013441,0.005376,0.001344,0.002688,0.000000,0.006720,0.002688,0.010753,0.010753,0.005376,0.002688,0.008065,0.004032,0.013441,0.009409,0.000000,0.002688,0.004032,0.000000,0.001344,0.001344,0.000000,0.002688,0.009409,0.002688,0.001344,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.130726,0.036872,0.043575,0.052514,0.024581,0.020112,0.026816,0.012291,0.003352,0.010056,0.000000,0.001117,0.003352,0.001117,0.003352,0.007821,0.005587,0.011173,0.010056,0.001117,0.000000,0.000000,0.001117,0.002235,0.001117,0.001117,0.001117,0.000000,0.000000,0.001117,0.002235,0.000000,0.012291,0.002235,0.005587,0.005587,0.003352,0.002235,0.001117,0.006704,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.037846,0.032023,0.000000,0.018923,0.020378,0.018923,0.008734,0.014556,0.014556,0.018923,0.008734,0.023290,0.024745,0.001456,0.000000,0.002911,0.004367,0.004367,0.000000,0.000000,0.004367,0.000000,0.010189,0.010189,0.002911,0.008734,0.008734,0.000000,0.002911,0.000000,0.001456,0.000000,0.000000,0.000000,0.002911,0.000000,0.001456,0.001456,0.007278,0.004367,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.103175,0.035714,0.043651,0.023810,0.031746,0.019841,0.039683,0.007937,0.003968,0.015873,0.000000,0.007937,0.003968,0.000000,0.003968,0.007937,0.003968,0.007937,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.003968,0.000000,0.000000,0.000000,0.000000,0.000000,0.003968,0.019841,0.000000,0.003968,0.011905,0.003968,0.000000,0.003968,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0.064057,0.035587,0.019573,0.039146,0.003559,0.012456,0.003559,0.012456,0.010676,0.035587,0.028470,0.000000,0.003559,0.000000,0.012456,0.001779,0.000000,0.000000,0.003559,0.000000,0.007117,0.000000,0.008897,0.003559,0.000000,0.000000,0.019573,0.012456,0.003559,0.000000,0.003559,0.014235,0.000000,0.001779,0.000000,0.000000,0.010676,0.003559,0.000000,0.00533

In [ ]:
freq_bgw_test=freq_bgw(test_df,list(freq_words_training['Word']))
freq_bgw_test= pd.DataFrame(freq_bgw_test, columns=freq_words_training['Word'])
freq_bgw_test
freq_bgw_test.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw_test.txt', header=True, index=None, sep='\t')

100%|██████████| 500/500 [06:55<00:00,  1.20it/s]


In [ ]:
freq_bgw_test=pd.read_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/freq-bgw_test.txt', sep='\t')
freq_bgw_test

,the,and,was,of,to,a,with,in,is,patient,no,she,for,he,were,on,this,at,then,right,or,left,has,as,that,her,history,there,had,his,be,normal,procedure,not,an,placed,1,are,well,i,...,lorcet,atheromatous,wasting,somnolence,tremendous,phisohex,immigrated,itch,anticoagulated,stranded,durable,precarinal,100s,cognition,850,participates,condylectomy,serve,officer,monocular,hematopoietic,talofibular,178,conquer,hydrate,sealing,pericranial,glandular,subconjunctival,rinse,cousin,photodocumentation,emphasized,interaction,stabilizing,radiolucent,l5s,proteins,equivalents,suv
0,0.048673,0.039823,0.022124,0.039823,0.026549,0.004425,0.008850,0.004425,0.004425,0.022124,0.000000,0.000000,0.013274,0.004425,0.004425,0.013274,0.000000,0.000000,0.000000,0.022124,0.000000,0.000000,0.000000,0.004425,0.000000,0.000000,0.008850,0.000000,0.008850,0.00885,0.000000,0.000000,0.004425,0.000000,0.004425,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.090909,0.030948,0.040619,0.034816,0.021277,0.009671,0.021277,0.017408,0.007737,0.015474,0.001934,0.001934,0.007737,0.000000,0.003868,0.009671,0.001934,0.000000,0.015474,0.000000,0.000000,0.017408,0.007737,0.007737,0.001934,0.001934,0.001934,0.001934,0.000000,0.00000,0.003868,0.000000,0.005803,0.000000,0.007737,0.003868,0.003868,0.000000,0.003868,0.001934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.039568,0.032374,0.025180,0.035971,0.007194,0.035971,0.014388,0.010791,0.017986,0.003597,0.000000,0.025180,0.010791,0.000000,0.000000,0.010791,0.007194,0.010791,0.000000,0.000000,0.000000,0.000000,0.003597,0.000000,0.003597,0.000000,0.014388,0.003597,0.003597,0.00000,0.003597,0.007194,0.003597,0.000000,0.007194,0.000000,0.007194,0.000000,0.007194,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.081290,0.042581,0.064516,0.014194,0.012903,0.014194,0.014194,0.023226,0.002581,0.015484,0.002581,0.003871,0.006452,0.000000,0.010323,0.010323,0.009032,0.001290,0.002581,0.011613,0.000000,0.001290,0.000000,0.016774,0.000000,0.003871,0.000000,0.003871,0.002581,0.00000,0.000000,0.000000,0.005161,0.000000,0.001290,0.015484,0.000000,0.000000,0.014194,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.033088,0.029412,0.003676,0.025735,0.029412,0.007353,0.003676,0.018382,0.011029,0.000000,0.014706,0.025735,0.003676,0.000000,0.000000,0.007353,0.003676,0.003676,0.000000,0.003676,0.007353,0.018382,0.014706,0.011029,0.011029,0.025735,0.000000,0.000000,0.000000,0.00000,0.000000,0.003676,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003676,0.007353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.037791,0.037791,0.042151,0.021802,0.017442,0.029070,0.018895,0.007267,0.010174,0.004360,0.010174,0.023256,0.008721,0.000000,0.010174,0.017442,0.014535,0.015988,0.001453,0.005814,0.001453,0.000000,0.001453,0.007267,0.005814,0.002907,0.001453,0.005814,0.002907,0.00000,0.005814,0.004360,0.001453,0.001453,0.004360,0.002907,0.001453,0.004360,0.004360,0.004360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
from scipy.sparse import csr_matrix

freq_bgw_scrMatrix=csr_matrix(freq_bgw.values)
freq_bgw_scrMatrix

<4000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3511981 stored elements in Compressed Sparse Row format>

In [ ]:
import tqdm

def bgw_transform(vocabulary,df):
  bgw=[]
  freq_word=list(vocabulary['Word'])
  nbr_word=vocabulary['Word'].shape[0]
  for row in tqdm.tqdm(df["words"]):
    bg=[]
    for word in row:
      if word in freq_word:
        bg.append(int(vocabulary[vocabulary['Word']==word]['id']))
    bg1=' '.join(map(str, bg))
    bgw.append(bg1)
  return(bgw)



In [ ]:
medical_text_train=bgw_transform(freq_words_training, train_df)
medical_text_train= pd.DataFrame(medical_text_train)
medical_text_train[1]=train_df['label']
medical_text_train.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/medical_text-train.txt', header=False, index=None, sep='\t')

100%|██████████| 4000/4000 [46:18<00:00,  1.44it/s]


In [ ]:
medical_text_valid=bgw_transform(freq_words_training, valid_df)
medical_text_valid= pd.DataFrame(medical_text_valid)
medical_text_valid[1]=valid_df['label']
medical_text_valid.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/medical_text-valid.txt', header=False, index=None, sep='\t')

100%|██████████| 499/499 [05:51<00:00,  1.42it/s]


In [ ]:
medical_text_test=bgw_transform(freq_words_training, test_df)
medical_text_test= pd.DataFrame(medical_text_test)
medical_text_test[1]=test_df['label']
medical_text_test.to_csv(r'/content/drive/MyDrive/Automn 2021/Machin Learning/Assignment3/medical_text-test.txt', header=False, index=None, sep='\t')

100%|██████████| 500/500 [05:57<00:00,  1.40it/s]


In [ ]:
import sklearn
import numpy as np
from sklearn.metrics import f1_score
from sklearn.naive_bayes import  BernoulliNB
from sklearn.naive_bayes import  GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import PredefinedSplit

In [ ]:
#merge train and validation dataset 
train_df_total_label=list(pd.concat([train_df['label'],valid_df['label']], ignore_index=True))
binary_bgw_total=pd.concat([binary_bgw_train,binary_bgw_valid], ignore_index=True)
binary_bgw_total['label']=train_df_total_label
# train_df_total

In [ ]:
#merge train and validation dataset 
train_df_total_label=list(pd.concat([train_df['label'],valid_df['label']], ignore_index=True))
freq_bgw_total=pd.concat([freq_bgw_train,freq_bgw_valid], ignore_index=True)
freq_bgw_total['label']=train_df_total_label
# train_df_total

# 2.a

In [ ]:
#labeling based on uniformly random class
import random
def random_labeling(N):
  label_pred=[]
  for i in range (N):
    label=0
    x=random.uniform(0, 1)
    if 0<=x<=0.25:
      label=1
    elif 0.25<x<=0.5:
      label=2
    elif 0.5<x<=0.75:
      label=3
    elif 0.75<x<=1:
      label=4
    label_pred.append(label)
  return label_pred
  

In [ ]:
label_random_train=random_labeling(train_df['label'].shape[0])
label_random_valid=random_labeling(valid_df['label'].shape[0])
label_random_test=random_labeling(test_df['label'].shape[0])

In [ ]:
def calc_results(pred_class, true_class):
  rnd_result={}
  for i in set(true_class):
    metric={'TP':0,'TN':0,'FP':0,'FN':0}
    for j,label in enumerate(pred_class):
      if label==true_class[j]==i:
        metric['TP']+=1
      elif label!=true_class[j]==i:
        metric['FN']+=1
      elif true_class[j]!=label==i:
        metric['FP']+=1
      else:
        metric['TN']+=1
    rnd_result[i]=metric
  return rnd_result

In [ ]:
calc_results(label_random_train, list(train_df['label']))

{1: {'FN': 955, 'FP': 640, 'TN': 2083, 'TP': 322},
 2: {'FN': 736, 'FP': 744, 'TN': 2248, 'TP': 272},
 3: {'FN': 719, 'FP': 778, 'TN': 2288, 'TP': 215},
 4: {'FN': 575, 'FP': 823, 'TN': 2396, 'TP': 206}}

In [ ]:
f1_train = f1_score( list(train_df['label']),label_random_train, average='macro')
f1_valid = f1_score( list(valid_df['label']),label_random_valid, average='macro')
f1_test = f1_score( list(test_df['label']),label_random_test, average='macro')
print(f'F1-score of random labeling\n Training: {f1_train}\n Vlidation: {f1_valid} \n Test:{f1_test}')

F1-score of random labeling
 Training: 0.25179305078177494
 Vlidation: 0.2643624871062423 
 Test:0.24236153846045608


In [ ]:
#labeling based on uniformly majority class
label_count_train=train_df.groupby(train_df['label']).agg({'words':['count'] })
label_count_train.columns=['label_count']
label_count_train=label_count_train.reset_index()
majority_class=int(label_count_train.loc[label_count_train['label_count']==label_count_train['label_count'].max()]['label'])
print(label_count_train)
print(f'The majority class for Medical dataset is: {majority_class}')

   label  label_count
0      1         1277
1      2         1008
2      3          934
3      4          781
The majority class for Medical dataset is: 1


In [ ]:
label_maj_train=[majority_class]*train_df['label'].shape[0]
label_maj_valid=[majority_class]*valid_df['label'].shape[0]
label_maj_test=[majority_class]*test_df['label'].shape[0]

In [ ]:
calc_results(label_maj_train, list(train_df['label']))

{1: {'FN': 0, 'FP': 2723, 'TN': 0, 'TP': 1277},
 2: {'FN': 1008, 'FP': 0, 'TN': 2992, 'TP': 0},
 3: {'FN': 934, 'FP': 0, 'TN': 3066, 'TP': 0},
 4: {'FN': 781, 'FP': 0, 'TN': 3219, 'TP': 0}}

In [ ]:
f1_train = f1_score( list(train_df['label']),label_maj_train, average='macro')
f1_valid = f1_score( list(valid_df['label']),label_maj_valid, average='macro')
f1_test = f1_score( list(test_df['label']),label_maj_test, average='macro')
print(f'F1-score of majority class labeling\n Training: {f1_train}\n Vlidation: {f1_valid} \n Test:{f1_test}')

F1-score of majority class labeling
 Training: 0.120996778472617
 Vlidation: 0.12424698795180723 
 Test:0.14183381088825217


# 2.b-Bernouli Naive Bayes

In the case of text classification, word occurrence vectors (rather than word count vectors) may be used to train and use this classifier. BernoulliNB might perform better on some datasets, especially those with shorter documents. It is advisable to evaluate both models, if time permits.

In [ ]:
# import numpy as np
# train_total = train_df_total.to_numpy()

In [ ]:
clf = BernoulliNB()
clf.fit(binary_bgw_train, train_df['label'])
f1_train = f1_score( list(train_df['label']),clf.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf.predict(binary_bgw_test), average='macro')
print(f'F1-score of  Bernouli Naive Bayes without hyperparameter tunning\n Training: {f1_train}\n Vlidation: {f1_valid} \n Test:{f1_test}')



F1-score of  Bernouli Naive Bayes without hyperparameter tunning
 Training: 0.5335748842966954
 Vlidation: 0.4586511509873923 
 Test:0.46750737765273176


In [ ]:
# clf = BernoulliNB()
# clf.fit(train_df_total.iloc[:,:10000], train_df_total['label'])
params = {'alpha': np.arange(0, 1.005, 0.005)}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_bnb = GridSearchCV(BernoulliNB(), param_grid=params, cv=p, scoring='f1_macro')
clf_bnb.fit(binary_bgw_total.iloc[:,:10000], binary_bgw_total['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None,
                                   fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.   , 0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 ,
       0.045, 0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085,
       0.09 , 0.095, 0.1  , 0.105, 0.11 , 0.115, 0.12 ,...
       0.765, 0.77 , 0.775, 0.78 , 0.785, 0.79 , 0.795, 0.8  , 0.805,
       0.81 , 0.815, 0.82 , 0.825, 0.83 , 0.835, 0.84 , 0.845, 0.85 ,
       0.855, 0.86 , 0.865, 0.87 , 0.875, 0.88 , 0.885, 0.89 , 0.895,
       0.9  , 0.905, 0.91 , 0.915, 0.92 , 0.925, 0.93 , 0.935, 0.94 ,
       0.945, 0.95 , 0.955, 0.96 , 0.965, 0.97 , 0.975, 0.98 , 0.985,
       0.99 , 0.995, 1.   ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', 

In [ ]:
clf_bnb.best_params_

{'alpha': 0.8150000000000001}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_bnb.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_bnb.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_bnb.predict(binary_bgw_test), average='macro')
print(f'F1-score of  Bernouli Naive Bayes with alpha={clf_bnb.best_params_}\n Training: {f1_train}\n Vlidation: {f1_valid} \n Test:{f1_test}')

F1-score of  Bernouli Naive Bayes with alpha={'alpha': 0.8150000000000001}
 Training: 0.5318753352433206
 Vlidation: 0.49798012032813965 
 Test:0.47337876955592384


# 2.b Decision Trees

In [ ]:
clf_dec=DecisionTreeClassifier()
clf_dec.fit(binary_bgw_train, train_df['label'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_dec.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_dec.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_dec.predict(binary_bgw_test), average='macro')
print(f'F1-score of Decision Tree without hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Decision Tree without hyperparameter tunning
 Training: 0.907376931421898
 Validation: 0.7096489149799332 
 Test:0.740212028565679


In [ ]:
# params = {'max_depth': np.arange(17, 20),'ccp_alpha': np.arange(0, .005, .001), 'max_features': np.arange(6000, 10001, 1000), 'max_leaf_nodes': np.arange(1500, 3000, 250), 'min_samples_leaf': np.arange(3, 7)}
params = {'ccp_alpha': np.arange(0, .005, .001), 'min_samples_leaf': np.arange(3, 10)}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_dec_h = GridSearchCV(DecisionTreeClassifier(), param_grid=params, cv=p, scoring='f1_macro')
clf_dec_h.fit(binary_bgw_total.iloc[:,:10000], binary_bgw_total['label'])

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
           

In [ ]:
clf_dec_h.best_params_

{'ccp_alpha': 0.001, 'min_samples_leaf': 6}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_dec_h.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_dec_h.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_dec_h.predict(binary_bgw_test), average='macro')
print(f'F1-score of Decision Tree with hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Decision Tree with hyperparameter tunning
 Training: 0.8089879709867438
 Validation: 0.8010233707952481 
 Test:0.798325636663274


# 2.b Logistic regression

In [ ]:
clf_lr = LogisticRegression().fit(binary_bgw_train, train_df['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
f1_train = f1_score( list(train_df['label']),clf_lr.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_lr.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_lr.predict(binary_bgw_test), average='macro')
print(f'F1-score of Logistic Regression without hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Logistic Regression without hyperparameter tunning
 Training: 0.9083094313212405
 Validation: 0.6851153427007235 
 Test:0.722191909688582


In [ ]:
params = {'penalty':['l1','l2'], 'C':np.arange(0.2, 0.6, .2), 'solver':['liblinear', 'saga']}
# params = {'penalty':['l1'], 'solver':['liblinear'], 'C':np.arange(.2, .4, .1)}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_rl_h = GridSearchCV(LogisticRegression(), param_grid=params, cv=p, scoring='f1_macro')
clf_rl_h.fit(binary_bgw_total.iloc[:,:10000], binary_bgw_total['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([0.2, 0.4]), 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [ ]:
clf_rl_h.best_params_

{'C': 0.2, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_rl_h.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_rl_h.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_rl_h.predict(binary_bgw_test), average='macro')
print(f'F1-score of Logistic Regression with hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Logistic Regression with hyperparameter tunning
 Training: 0.8618431127019649
 Validation: 0.8572274533436299 
 Test:0.7940420793255759


# 2.b Linear SVM

In [ ]:
clf_svm = LinearSVC().fit(binary_bgw_train, train_df['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
f1_train = f1_score( list(train_df['label']),clf_svm.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_svm.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_svm.predict(binary_bgw_test), average='macro')
print(f'F1-score of Linear SVM without hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Linear SVM without hyperparameter tunning
 Training: 0.9076197296518124
 Validation: 0.7415093629873268 
 Test:0.7776470979223433


In [ ]:
params = {'penalty':['l1','l2'], 'C':np.arange(0.1, 0.6, 0.1), 'loss':['hinge', 'squared_hinge'], 'dual':[True,False]}
# params = {'penalty':['l1', 'l2'], 'loss':['hinge', 'squared_hinge']}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_svm_h = GridSearchCV(LinearSVC(), param_grid=params, cv=p)
clf_svm_h.fit(binary_bgw_total.iloc[:,:10000], binary_bgw_total['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5]), 'dual': [False],
                         'loss': ['squared_hinge'], 'penalty': ['l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
clf_svm_h.best_params_

{'C': 0.1, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l1'}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_svm_h.predict(binary_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_svm_h.predict(binary_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_svm_h.predict(binary_bgw_test), average='macro')
print(f'F1-score of Linear SVM with hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Linear SVM with hyperparameter tunning
 Training: 0.8828166816174052
 Validation: 0.8873905876846092 
 Test:0.8001591294732682


# 3.a Naive Bayes

In [ ]:
clf_gnb = GaussianNB()
clf_gnb.fit(freq_bgw_train, train_df['label'])
f1_train = f1_score( list(train_df['label']),clf_gnb.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_gnb.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_gnb.predict(freq_bgw_test), average='macro')
print(f'F1-score of  Bernouli Naive Bayes for frequency BoW without hyperparameter tunning\n Training: {f1_train}\n Vlidation: {f1_valid} \n Test:{f1_test}')

F1-score of  Bernouli Naive Bayes for frequency BoW without hyperparameter tunning
 Training: 0.6924892955717259
 Vlidation: 0.3592475251688793 
 Test:0.35841623737894085


In [ ]:
# clf = BernoulliNB()
# clf.fit(train_df_total.iloc[:,:10000], train_df_total['label'])
params = {'var_smoothing': np.logspace(-11, 3, num=50)}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_gnb_h = GridSearchCV(GaussianNB(), param_grid=params, cv=p, scoring='f1_macro')
clf_gnb_h.fit(freq_bgw_total.iloc[:,:10000], freq_bgw_total['label'])

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=GaussianNB(priors=None, var_smoothing=1e-09),
             iid='deprecated', n_jobs=None,
             param_grid={'var_smoothing': array([1.00000000e-11, 1.93069773e-11, 3.72759372e-11, 7.19685673e-11,
       1.38949549e-10, 2.68269580e-10, 5.17947468e-10, 1.00000000e-09,
       1.93069773e-09, 3.72759372e-09,...
       1.38949549e-02, 2.68269580e-02, 5.17947468e-02, 1.00000000e-01,
       1.93069773e-01, 3.72759372e-01, 7.19685673e-01, 1.38949549e+00,
       2.68269580e+00, 5.17947468e+00, 1.00000000e+01, 1.93069773e+01,
       3.72759372e+01, 7.19685673e+01, 1.38949549e+02, 2.68269580e+02,
       5.17947468e+02, 1.00000000e+03])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [ ]:
clf_gnb_h.best_params_

{'var_smoothing': 0.001}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_gnb_h.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_gnb_h.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_gnb_h.predict(freq_bgw_test), average='macro')
print(f'F1-score of  Gaussian Naive Bayes for frequency BoW with smoothing var.={clf_gnb_h.best_params_}\n Training: {f1_train}\n Vlidation: {f1_valid} \n Test:{f1_test}')

F1-score of  Gaussian Naive Bayes for frequency BoW with smoothing var.={'var_smoothing': 0.001}
 Training: 0.5769039084787
 Vlidation: 0.5449155234246417 
 Test:0.45004783517410624


# 3.a Decision Trees

In [ ]:
clf_dec_f=DecisionTreeClassifier()
clf_dec_f.fit(freq_bgw_train, train_df['label'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_dec_f.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_dec_f.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_dec_f.predict(freq_bgw_test), average='macro')
print(f'F1-score of Decision Tree for Frequency BoW without hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Decision Tree for Frequency BoW without hyperparameter tunning
 Training: 0.9084872405754333
 Validation: 0.7074749537410122 
 Test:0.7013769667229854


In [ ]:
# params = {'max_depth': np.arange(14, 19), 'max_features': np.arange(6000, 10001, 1000), 'max_leaf_nodes': np.arange(1500, 2500, 250), 'min_samples_leaf': np.arange(2, 6)}
# params = {'ccp_alpha': np.arange(0, .005, .001),'max_depth': np.arange(14, 18), 'max_features': np.arange(7000, 10001, 1000), 'max_leaf_nodes': np.arange(1500, 2500, 250), 'min_samples_leaf': np.arange(1, 5)}
params = {'ccp_alpha': np.arange(0, .005, .001)}
f = [-1 for i in range(train_df.shape[0])] + [0 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_dec_f_h = GridSearchCV(DecisionTreeClassifier(), param_grid=params, cv=p)
clf_dec_f_h.fit(freq_bgw_total.iloc[:,:10000], freq_bgw_total['label'])

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
           

In [ ]:
clf_dec_f_h.best_params_

{'ccp_alpha': 0.002}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_dec_f_h.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_dec_f_h.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_dec_f_h.predict(freq_bgw_test), average='macro')
print(f'F1-score of Decision Tree for frequency BoW with hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Decision Tree for frequency BoW with hyperparameter tunning
 Training: 0.7733022429864261
 Validation: 0.7860427446769105 
 Test:0.7671615917065018


# 3.a Logistic regression

In [ ]:
clf_lr_f = LogisticRegression().fit(freq_bgw_train, train_df['label'])

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_lr_f.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_lr_f.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_lr_f.predict(freq_bgw_test), average='macro')
print(f'F1-score of Logistic Regression for frequency BoW without hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Logistic Regression for frequency BoW without hyperparameter tunning
 Training: 0.3325629904424544
 Validation: 0.32519120594231016 
 Test:0.32947115540038147


In [ ]:
import numpy as np
params = {'penalty':['l1','l2'], 'C':np.arange(40, 100, 5), 'solver':['liblinear', 'saga']}
# params = {'penalty':['l1'], 'solver':['liblinear'], 'C':np.arange(.2, .4, .1)}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_lr_f_h = GridSearchCV(LogisticRegression(), param_grid=params, cv=p, scoring='f1_macro')
clf_lr_f_h.fit(freq_bgw_total.iloc[:,:10000], freq_bgw_total['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             

In [ ]:
clf_lr_f_h.best_params_

{'C': 75, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_lr_f_h.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_lr_f_h.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_lr_f_h.predict(freq_bgw_test), average='macro')
print(f'F1-score of Logistic Regression for frequency BoW with hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Logistic Regression for frequency BoW with hyperparameter tunning
 Training: 0.8574325884731568
 Validation: 0.8610039148141776 
 Test:0.770223539335674


# 3.a Linear SVM

In [ ]:
clf_svm_f = LinearSVC().fit(freq_bgw_train, train_df['label'])

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_svm_f.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_svm_f.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_svm_f.predict(freq_bgw_test), average='macro')
print(f'F1-score of Linear SVM for frequency BoW without hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Linear SVM for frequency BoW without hyperparameter tunning
 Training: 0.4017287747626928
 Validation: 0.38635388240794466 
 Test:0.36746611986152194


In [ ]:
params = {'penalty':['l1','l2'], 'C':np.arange(200, 400, 10), 'loss':['hinge', 'squared_hinge'], 'dual':[True,False]}
# params = {'penalty':['l1'], 'C':np.arange(1, 100, 20), 'loss':['squared_hinge'], 'dual':['False'], 'random_state':[0]}
f = [-1 for i in range(train_df.shape[0])] + [1 for i in range(valid_df.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_svm_f_h = GridSearchCV(LinearSVC(), param_grid=params, cv=p)
clf_svm_f_h.fit(freq_bgw_total.iloc[:,:10000], freq_bgw_total['label'])

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True

 

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320,
       330, 340, 350, 360, 370, 380, 390]),
                         'dual': [True, False],
                         'loss': ['hinge', 'squared_hinge'],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
clf_svm_f_h.best_params_

{'C': 240, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l1'}

In [ ]:
f1_train = f1_score( list(train_df['label']),clf_svm_f_h.predict(freq_bgw_train), average='macro')
f1_valid = f1_score( list(valid_df['label']),clf_svm_f_h.predict(freq_bgw_valid), average='macro')
f1_test = f1_score( list(test_df['label']),clf_svm_f_h.predict(freq_bgw_test), average='macro')
print(f'F1-score of Linear SVM for frequency BoW with hyperparameter tunning\n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Linear SVM for frequency BoW with hyperparameter tunning
 Training: 0.8979681728512134
 Validation: 0.9072911625334804 
 Test:0.7583647448389872
